In [1]:
options = ["piedra", "tijeras", "papel"]

In [6]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
        
    return result

In [7]:
search_winner("papel", "tijeras")

2

In [9]:
test = [
    ["piedra", "piedra", 0],
    ["piedra", "tijeras", 1],
    ["piedra", "papel", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: piedra player2: piedra Winner: 0 Validation: 0
player1: piedra player2: tijeras Winner: 1 Validation: 1
player1: piedra player2: papel Winner: 2 Validation: 2


In [10]:
from random import choice
def get_choice():
    return choice(options)


In [11]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: piedra player2: piedra Winner: 0 
player1: tijeras player2: tijeras Winner: 0 
player1: piedra player2: piedra Winner: 0 
player1: papel player2: tijeras Winner: 2 
player1: papel player2: papel Winner: 0 
player1: piedra player2: piedra Winner: 0 
player1: tijeras player2: piedra Winner: 2 
player1: piedra player2: tijeras Winner: 1 
player1: piedra player2: tijeras Winner: 1 
player1: papel player2: papel Winner: 0 


In [13]:
def str_to_list(option):
    if option=="piedra":
        res = [1,0,0]
    elif option=="tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ["piedra", "tijeras", "papel"]))
data_y = list(map(str_to_list, ["papel", "piedra", "tijeras"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [14]:
from sklearn.neural_network import MLPClassifier


In [15]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [16]:
model = clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True)


/Users/david/Library/Python/3.6/lib/python/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [17]:
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y


        

In [20]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)


Player1: tijeras Player2 (modelo): [ 0.11859622  0.12092568  0.84820899] --> piedra
Comprobamos: p1 VS p2: 2
[[0, 1, 0]]
[[1, 0, 0]]
Score: {'win': 1, 'loose': 0} 100.0 %


In [22]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break


Iter: 1 - score: {'win': 546, 'loose': 454} 54.6 %
Iter: 2 - score: {'win': 573, 'loose': 427} 57.3 %
Iter: 3 - score: {'win': 563, 'loose': 437} 56.3 %
Iter: 4 - score: {'win': 514, 'loose': 486} 51.4 %
Iter: 5 - score: {'win': 549, 'loose': 451} 54.9 %
Iter: 6 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 7 - score: {'win': 350, 'loose': 650} 35.0 %
Iter: 8 - score: {'win': 348, 'loose': 652} 34.8 %
Iter: 9 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 10 - score: {'win': 313, 'loose': 687} 31.3 %
Iter: 11 - score: {'win': 349, 'loose': 651} 34.9 %
Iter: 12 - score: {'win': 316, 'loose': 684} 31.6 %
Iter: 13 - score: {'win': 315, 'loose': 685} 31.5 %
Iter: 14 - score: {'win': 314, 'loose': 686} 31.4 %
Iter: 15 - score: {'win': 355, 'loose': 645} 35.5 %
Iter: 16 - score: {'win': 342, 'loose': 658} 34.2 %
Iter: 17 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 18 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 19 - score: {'win': 343, 'loose': 657} 34.3 %
Iter: 20 - score: {'w

Iter: 159 - score: {'win': 538, 'loose': 462} 53.8 %
Iter: 160 - score: {'win': 576, 'loose': 424} 57.6 %
Iter: 161 - score: {'win': 574, 'loose': 426} 57.4 %
Iter: 162 - score: {'win': 549, 'loose': 451} 54.9 %
Iter: 163 - score: {'win': 563, 'loose': 437} 56.3 %
Iter: 164 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 165 - score: {'win': 558, 'loose': 442} 55.8 %
Iter: 166 - score: {'win': 534, 'loose': 466} 53.4 %
Iter: 167 - score: {'win': 560, 'loose': 440} 56.0 %
Iter: 168 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 169 - score: {'win': 546, 'loose': 454} 54.6 %
Iter: 170 - score: {'win': 555, 'loose': 445} 55.5 %
Iter: 171 - score: {'win': 563, 'loose': 437} 56.3 %
Iter: 172 - score: {'win': 543, 'loose': 457} 54.3 %
Iter: 173 - score: {'win': 581, 'loose': 419} 58.1 %
Iter: 174 - score: {'win': 554, 'loose': 446} 55.4 %
Iter: 175 - score: {'win': 551, 'loose': 449} 55.1 %
Iter: 176 - score: {'win': 535, 'loose': 465} 53.5 %
Iter: 177 - score: {'win': 563, 'loose': 437} 

In [23]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()


Loading BokehJS ...

In [24]:
x = range(len(historic_pct))
y = historic_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900)

p.line(x, y, legend=None, line_width=1)
show(p)


In [25]:
model.predict_proba([str_to_list("piedra")])

array([[ 0.00829449,  0.0049859 ,  0.98453702]])